# Graduate Rotational Internship Program - The Sparks Foundation
## Computer Vision & Internet of Things - Detection of face mask (real time)
## Submitted by - <font color=blue>Shvetha M Nambiar</font> 

## Code:
### Import libraries

In [1]:
import numpy as np
from PIL import UnidentifiedImageError
import PIL
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense,Dropout
from keras.models import Model, load_model
from keras.callbacks import TensorBoard, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
import imutils

Using TensorFlow backend.


### Create the CNN model

In [2]:
model =Sequential([
    Conv2D(100, (3,3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2,2),
    
    Conv2D(100, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    
    Flatten(),
    Dropout(0.5),
    Dense(50, activation='relu'),
    Dense(2, activation='softmax')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

### Upload and augment training dataset images

In [3]:
TRAINING_DIR = "C:/Users/Shvetha M Nambiar/Desktop/Vit files/GRIP foundation task/dataset"
train_datagen = ImageDataGenerator(rescale=1.0/255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(TRAINING_DIR, 
                                                    batch_size=10, 
                                                    target_size=(150, 150))

Found 1376 images belonging to 2 classes.


### Upload test dataset images

In [4]:
VALIDATION_DIR = "C:/Users/Shvetha M Nambiar/Desktop/Vit files/GRIP foundation task/test"
validation_datagen = ImageDataGenerator(rescale=1.0/255)

validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR, 
                                                         batch_size=10, 
                                                         target_size=(150, 150))
checkpoint = ModelCheckpoint('model2-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')

Found 194 images belonging to 2 classes.


### Train model 

In [5]:
newmodel_1 = model.fit_generator(train_generator,
                              epochs=10,
                              validation_data=validation_generator,
                              callbacks=[checkpoint])

Epoch 1/10
138/138 [==============================] - 133s 966ms/step - loss: 0.3585 - acc: 0.8561 - val_loss: 0.5335 - val_acc: 0.9072
Epoch 2/10
138/138 [==============================] - 131s 952ms/step - loss: 0.1926 - acc: 0.9346 - val_loss: 0.1486 - val_acc: 0.9433
Epoch 3/10
138/138 [==============================] - 132s 954ms/step - loss: 0.2013 - acc: 0.9237 - val_loss: 0.0123 - val_acc: 0.9639
Epoch 4/10
138/138 [==============================] - 132s 955ms/step - loss: 0.1587 - acc: 0.9448 - val_loss: 0.0109 - val_acc: 0.9691
Epoch 5/10
138/138 [==============================] - 131s 951ms/step - loss: 0.1790 - acc: 0.9353 - val_loss: 0.0525 - val_acc: 0.9742
Epoch 6/10
138/138 [==============================] - 118s 854ms/step - loss: 0.2247 - acc: 0.9222 - val_loss: 0.3731 - val_acc: 0.9536
Epoch 7/10
138/138 [==============================] - 103s 744ms/step - loss: 0.1314 - acc: 0.9557 - val_loss: 0.0132 - val_acc: 0.9639
Epoch 8/10
138/138 [============================

### Save the trained model

In [6]:
model.save("newmodel_1.h")

### Load trained model if already saved

In [7]:
from keras.models import load_model
model = load_model('C:/Users/Shvetha M Nambiar/Spyder/Spyder projects/Spark foundation/newmodel_1.h')

### Summary of the created CNN model

In [8]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 148, 148, 100)     2800      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 100)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 72, 72, 100)       90100     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 36, 36, 100)       0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 129600)            0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 129600)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 50)               

### Testing the model
### Real-Time mask detection

In [ ]:
labels_dict={0:'without mask',1:'mask'}
color_dict={0:(0,0,255),1:(0,255,0)}

size = 4
webcam = cv2.VideoCapture(0) #Use camera 0
# We load the xml file
classifier = cv2.CascadeClassifier('D:/anaconda/pkgs/libopencv-4.4.0-py37_2/Library/etc/haarcascades/haarcascade_frontalface_default.xml')

while True:
    (rval, im) = webcam.read()
    im=cv2.flip(im,1,1) #Flip to act as a mirror

    # Resize the image to speed up detection
    mini = cv2.resize(im, (im.shape[1] // size, im.shape[0] // size))

    # detect MultiScale / faces 
    faces = classifier.detectMultiScale(mini)

    # Draw rectangles around each face
    for f in faces:
        (x, y, w, h) = [v * size for v in f] #Scale the shapesize backup
        #Save just the rectangle faces in SubRecFaces
        face_img = im[y:y+h, x:x+w]
        resized=cv2.resize(face_img,(150,150))
        normalized=resized/255.0
        reshaped=np.reshape(normalized,(1,150,150,3))
        reshaped = np.vstack([reshaped])
        result=model.predict(reshaped)
        #print(result)
        
        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(im,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(im,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(im, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        
    # Show the image
    cv2.imshow('LIVE',   im)
    key = cv2.waitKey(10)
    # if Esc key is press then break out of the loop 
    if key == 27: #The Esc key
        break
# Stop video
webcam.release()

# Close all started windows
cv2.destroyAllWindows()